In [2]:
from graph import GraphDB
import random
import numpy as np
import re
import pandas as pd
import Embedding_Model as em

[nltk_data] Downloading package wordnet to C:\Users\Andre
[nltk_data]     Italiano\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Andre
[nltk_data]     Italiano\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Andre Italiano\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
model = em.Sentence_transformer()

In [4]:
type(["what does labelled data look like again?","I have no idea"])

list

In [5]:
example= model(["what does labelled data look like again?","I have no idea"])
print(len(example))

okay
768


In [6]:
example= model("what does labelled data look like again?")
print(example)

[-9.33731813e-03 -1.64579712e-02 -4.59779566e-03  4.13153060e-02
  2.47797370e-03  2.22577294e-03  2.97685862e-02  1.47979334e-02
  3.74165969e-03  3.54732503e-03 -3.44138891e-02  3.94662051e-03
  2.73811887e-03  5.65520190e-02  5.33628929e-03 -6.11356348e-02
  7.55072385e-03 -2.70124581e-02 -1.58088021e-02  4.44912910e-03
 -1.94677673e-02 -3.54028717e-02 -7.70181464e-03  1.05398931e-02
 -5.76259419e-02 -7.85914063e-03 -3.51283909e-03 -5.79385785e-03
 -8.01106635e-03 -7.31172040e-02  2.06505451e-02  5.73195517e-02
  1.99892372e-02 -3.27400155e-02  1.57998454e-06 -6.52447576e-03
 -1.20986467e-02 -1.21410042e-02 -2.96697244e-02  5.29183215e-03
 -1.38092665e-02  2.04523094e-02  2.99224835e-02 -2.76091248e-02
  2.77317483e-02 -9.30247232e-02  1.19371163e-02  1.74298193e-02
  1.56428013e-02 -3.00717708e-02  6.94331154e-03 -2.81266309e-02
 -4.89965454e-02 -4.08873195e-03  1.12356819e-01  9.73439962e-03
  1.26178758e-02  4.96633761e-02  6.18573353e-02  8.40534493e-02
 -1.58706997e-02  3.84977

In [7]:
# getting data 
date_data_path = r'../../../data/tender_postcode.xlsx'

path = r'../../../data/embedding_data/'
embedding_data = np.load(path+"sent_transformer_embeddings.npz")
embedding_data = [list(embedding_data[f]) for f in embedding_data.files]

tender_references_raw = ''
tender_titles_raw = ''

with open(path+'tender_references.txt', 'r') as file:
    for line in file:
        tender_references_raw += line
        
with open(path+"tender_titles.txt",'r') as file:
    for line in file:
        tender_titles_raw += line
        
        
for item in ['"',"'",'[',']'," "]:
    tender_references_raw = tender_references_raw.replace(item, "")
tender_references = tender_references_raw.split(",")

tender_titles = []
for item in tender_titles_raw.split(","):
    tender_titles.append(re.sub(r'[\[\]/\\\'"]', '', item))

coordinate_data = np.load(path+"tender_coordinates.npz")
coordinate_data = [list(coordinate_data[f]) for f in coordinate_data]

expire_dates = pd.read_excel(date_data_path)
expire_dates = list(expire_dates['Expiry Date'])

In [8]:
# save to dictionary format
embedding_dictionary = {}
for i in range(len(tender_references)):
    embedding_dictionary[tender_references[i]] = {"embedding":embedding_data[i],"titles":tender_titles[i],"location":coordinate_data[i],"expire_date":expire_dates[i]}

In [9]:
graph_db = GraphDB(embedding_dictionary)

In [12]:
import folium
from folium import plugins
from folium.features import DivIcon
from ipywidgets import interact, widgets
import networkx as nx  

import pandas as pd
from datetime import datetime

start_date = datetime(2018, 1, 1)
end_date = datetime(2036, 1, 1)

dates = pd.date_range(start_date, end_date, freq='M')

options = [(date.strftime(' %b %Y '), date) for date in dates]
index = (0, len(options)-1)

selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Tender Expiration date ',
    orientation='horizontal',
    layout={'width': '500px'}
)
def get_node_positions(reference, num_nodes, max_distance,date1,date2):
    query_result = graph_db.closest(reference, num_nodes, max_distance,date1,date2)
    
    G = graph_db.draw_graph(query_result)
    
    return nx.get_node_attributes(G, "location"), nx.get_edge_attributes(G,"similarity")

def create_map(map_center):
    m = folium.Map(location=map_center, zoom_start=5)
    return m

def query_and_display(reference, num_nodes, max_distance,expire_date):
    edge_text_group = folium.FeatureGroup()
    node_positions,node_similarity = get_node_positions(reference, num_nodes, max_distance,expire_date[0],expire_date[1])
    map_center = node_positions[reference]    
    
    # create map with centered on reference 
    m = create_map(map_center)
    
    # add markers for ref positions on map
    for ref,position in node_positions.items():
        title = graph_db.data[ref]['titles']
        # plot ref marker
        folium.CircleMarker(location=position,popup=title,fill_color='blue').add_to(m)
        
        # plot ref label
        folium.Marker(location=position,popup=reference,icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12pt">{ref}</div>',
        )).add_to(m)
        
        if ref != reference:
            edge = folium.PolyLine([node_positions[reference],position],popup=str(node_similarity[(reference, ref)]))
            edge.add_to(m)
            edge_text = plugins.PolyLineTextPath(edge,text = str(node_similarity[(reference, ref)]),center=True,attributes={'fill': '007DEF', 'font-weight': 'bold', 'font-size': '24'})
            edge_text_group.add_child(edge_text)
            
      
    m.add_child(edge_text_group)

    display(m)

reference_input = widgets.Text(value='ARA201901891', description='Enter Reference:')
number_slider = widgets.IntSlider(value=2, min=1, max=10, step=1, description='Closest Tenders:')
distance_slider = widgets.FloatSlider(value=100, min=1, max=300, step=1, description='Max Distance:')
tender_expire_slider = widgets.FloatSlider(value=100, min=1, max=300, step=1, description='Tender Expiry Month:')
interact(query_and_display, reference=reference_input, num_nodes=number_slider, max_distance=distance_slider,expire_date=selection_range_slider)


interactive(children=(Text(value='ARA201901891', description='Enter Reference:'), IntSlider(value=2, descripti…

<function __main__.query_and_display(reference, num_nodes, max_distance, expire_date)>